In [1]:
## Importing libraries
import os
import sys
import json
import numpy as np
import modin.pandas as pd
import requests as rq
from snps import SNPs
import flatten_json as fj

In [2]:
## Constants of the application
os.environ["MODIN_ENGINE"] = "dask" 
REFSNP_API = 'https://api.ncbi.nlm.nih.gov/variation/v0/refsnp/{snp_id}/frequency'
OPENSNP_API = 'https://opensnp.org/data/{user_id}'

In [3]:
# Module functions

## Safe URL API requests from https://github.com/ncbi/dbsnp/blob/master/tutorials/Variation%20Services/Jupyter_Notebook/spdi_batch.ipynb
def api_request(url):
    try:
        r = rq.get(url)
    except rq.exceptions.Timeout:
        # Maybe set up for a retry, or continue in a retry loop
        print("ERROR: Timeout")
    except rq.exceptions.TooManyRedirects:
        # Tell the user their URL was bad and try a different one
        print("ERROR: bad url =" + url)
    except rq.exceptions.RequestException as e:
        # catastrophic error. bail.
        print(e)
        sys.exit(1)
    if (r.status_code == 200):
        return r
    else:
        print("ERROR: status code = " + str(r.status_code))
        return None

## Recovering the reference allele of the genotyped variant
def get_reference_allele_api(snp_id):
    var_dict = api_request(REFSNP_API.format(snp_id=snp_id)).json()
    allele_freqs = {}

    var_dict_flatten = fj.flatten(var_dict)
    for key, value in var_dict_flatten.items():
        if('allele_counts' in key):
            allele = key[-1]
            if(allele in allele_freqs.keys()):
                allele_freqs[allele] = allele_freqs[allele] + value
            else:
                allele_freqs[allele] = value

    major_allele = sorted(allele_freqs.items(), key=lambda item: item[1])[-1][0]
    print("SNP: {snp_id} has a major allele of {major_allele}".format(snp_id=snp_id, major_allele=major_allele))
    return major_allele

## Recovering the genotype dataset from a given individual
def get_genotype_individual(user_id):
    if(user_id != None):
        raw_data = api_request(OPENSNP_API.format(user_id=user_id))
        indiv_snps = SNPs(raw_data.content)
        genotype = pd.DataFrame(indiv_snps.snps.reset_index())
    return genotype

## Extracting the genotyped alleles from each individual for the PRS
def get_genotyped_alleles_individual(user_genotype, model_table):
    merged_df = user_genotype.merge(model_table,left_on='rsid', right_on='snp_id', how='inner')
    return merged_df

## Takes a model table and extracts the 
def assign_reference_allele(snp_entry):
    snp_id = snp_entry['snp_id'].replace(r'rs', '')
    ref_allele = get_reference_allele_api(snp_id)
    return ref_allele

## Get minor allele dosage
def get_minor_allele_dosage(snp_entry):
    reference_allele = snp_entry['reference_allele']
    genotype = snp_entry['genotype']
    dosage = 2 - genotype.count(reference_allele)
    return dosage

## Obtaining the reference (no-effect) allele
def get_reference_alleles_model(model_table):
    model_table['reference_allele'] = model_table.apply(lambda x: assign_reference_allele(x), axis=1)
    return model_table

## Attaching the reference (no-effect) allele and the dosage of the minor (effect) allele
def merge_allelic_data(user_genotype, model_table):
    final_genotype_individual = get_genotyped_alleles_individual(user_genotype, model_table)
    final_genotype_individual['minor_allele_dosage'] = final_genotype_individual.apply(lambda x: get_minor_allele_dosage(x), axis=1)
    return final_genotype_individual

## Calculating the PRS from the effect size and the minor (effect) allele dosage
def calculating_prs_individual(final_genotype_individual):
    prs = final_genotype_individual['effect_size'] * final_genotype_individual['minor_allele_dosage']
    return sum(prs)
    

In [4]:
## Getting the individual genotypes
individuo = get_genotype_individual('11008.23andme.9131')

mod1 = pd.read_csv('datasets/model_a.tsv', sep='\t')
mod1 = get_reference_alleles_model(mod1)
geno_indiv_mod1 = merge_allelic_data(individuo, mod1)
prs_mod1 = calculating_prs_individual(geno_indiv_mod1)
print("The PRS of the individual {} for the Model A is: {prs}".format(prs=prs_mod1))

mod2 = pd.read_csv('datasets/model_b.tsv', sep='\t')
mod2 = get_reference_alleles_model(mod2)
geno_indiv_mod2 = merge_allelic_data(individuo, mod2)
prs_mod2 = calculating_prs_individual(geno_indiv_mod2)
print("The PRS for the Model B is: {prs}".format(prs=prs_mod2))

To remove this warning, run the following python code before doing dataframe operations:

    from distributed import Client

    client = Client()



The PRS for the Model A is: -0.6663467602183987
The PRS for the Model B is: -0.7662909745323819


In [6]:
geno_indiv_mod1

,rsid,chrom,pos,genotype,snp_id,effect_size,reference_allele,minor_allele_dosage
0,rs2001142,1,7348943,CC,rs2001142,0.008742,C,0
1,rs1009941,1,9275194,AG,rs1009941,-0.002960,A,1
2,rs2990678,1,13837213,CT,rs2990678,-0.006790,C,1
3,rs12751422,1,27493679,TT,rs12751422,-0.006500,C,2
4,rs11583644,1,30419370,AG,rs11583644,0.013899,A,1
...,...,...,...,...,...,...,...,...
196,rs4811971,20,56796784,GG,rs4811971,0.026296,A,2
197,rs1918388,21,23964448,AA,rs1918388,-0.010879,C,2
198,rs2827703,21,24143818,GG,rs2827703,-0.006051,G,0
199,rs4819996,22,17790731,AG,rs4819996,0.001344,G,1
